# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# Output Directory
output_data_loc = "../output_data"

# Output File (CSV)
output_data_file = f"{output_data_loc}/cities.csv"

# Read into DataFrame
city_weather_data_df = pd.read_csv(output_data_file)

# Render
city_weather_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Isangel,-19.5500,169.2667,69.48,81,45,8.52,VU,1632765468
1,1,Polewali,-3.4324,119.3435,76.05,89,71,3.36,ID,1632765694
2,2,Grand Gaube,-20.0064,57.6608,72.37,71,3,8.99,MU,1632765490
3,3,Anadyr,64.7500,177.4833,35.01,80,81,5.99,RU,1632765694
4,4,Puerto Ayora,-0.7393,-90.3518,70.20,86,96,10.00,EC,1632765694


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key=g_key)

In [19]:
# Convert data frame fields to values gmaps can use
locations = city_weather_data_df[["Lat", "Lng"]].astype(float)
humidity = city_weather_data_df["Humidity"].astype(float)

# create a figure
fig = gmaps.figure()

# Create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
# Add it
fig.add_layer(heat_layer)

# Render
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# Drop nulls
city_weather_data_df = city_weather_data_df.dropna()

# Filter to reduce result set for ideal weather
mask = (city_weather_data_df['Max Temp']>=70) \
& (city_weather_data_df['Max Temp']<=80) \
& (city_weather_data_df['Wind Speed']<10) \
& (city_weather_data_df['Cloudiness']==0)

# Render
hotel_df = city_weather_data_df.loc[mask].copy()
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,Mahon,39.8885,4.2658,75.76,73,0,4.61,ES,1632765759
129,129,Tagāb,34.8550,69.6492,72.90,50,0,2.37,AF,1632765822
262,262,Letlhakane,-21.4167,25.5833,76.57,12,0,5.77,BW,1632766001
280,280,Shenjiamen,29.9576,122.2980,75.22,82,0,4.81,CN,1632766054
289,289,Luanda,-8.8368,13.2343,75.20,83,0,8.05,AO,1632765778
384,384,Erzin,36.9559,36.2000,76.01,58,0,3.85,TR,1632766178


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df['Hotel Name'] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = { 
    "keyword":"hotel",
    "radius": 5000,    
    "key": g_key,
}

for index, row in hotel_df.iterrows():

    # add location to the params
    params['location'] = f"{row['Lat']},{row['Lng']}"

    # Make the request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract the results
    results = response['results']
    
    # Append the data frame
    try:
        print(f"Closest hotel is {results[0]['name']}.")        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']       
        
    except (KeyError, IndexError):
        print("No hotel result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = "**No Hotel info**" 

hotel_df

Retrieving Results for Index 69: Mahon.
Closest hotel is HOTEL VICTORI.
Retrieving Results for Index 129: Tagāb.
No hotel result... skipping.
Retrieving Results for Index 262: Letlhakane.
Closest hotel is Hotel Phudzi.
Retrieving Results for Index 280: Shenjiamen.
Closest hotel is Hilton Zhoushan.
Retrieving Results for Index 289: Luanda.
Closest hotel is EPIC SANA Luanda.
Retrieving Results for Index 384: Erzin.
Closest hotel is Hattusa Vacation Thermal Club Erzin.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
69,69,Mahon,39.8885,4.2658,75.76,73,0,4.61,ES,1632765759,HOTEL VICTORI
129,129,Tagāb,34.8550,69.6492,72.90,50,0,2.37,AF,1632765822,**No Hotel info**
262,262,Letlhakane,-21.4167,25.5833,76.57,12,0,5.77,BW,1632766001,Hotel Phudzi
280,280,Shenjiamen,29.9576,122.2980,75.22,82,0,4.81,CN,1632766054,Hilton Zhoushan
289,289,Luanda,-8.8368,13.2343,75.20,83,0,8.05,AO,1632765778,EPIC SANA Luanda
384,384,Erzin,36.9559,36.2000,76.01,58,0,3.85,TR,1632766178,Hattusa Vacation Thermal Club Erzin


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))